In [ ]:
# imports
import numpy as np
import xarray as xr
import pandas as pd
import proplot as pplt
import matplotlib.pyplot as plt

In [ ]:
### input data
startD='1979-01-01'
endD='2014-12-31'
Inpath='/home/jupyter-dipti/work/ISMR_CITCZ/Data/'
ERAInput=xr.open_dataset(Inpath+'Regrided_ERA5.nc').sel(time=slice(startD,endD))
# load list of models,members
modelList = pd.read_csv('Model_List.csv')

In [ ]:
#### functions to calculate different variables for ERA5 as well as cmip models

### define vars

ERAvarDefs={
        'netTOA':{'vars':['netTOA'],'signs':[1.0]},
        'netTOACRE':{'vars':['cres','crel'],'signs':[1.0,1.0]},
        'cres':{'vars':['cres'],'signs':[1.0]},
        'crel':{'vars':['crel'],'signs':[1.0]},
        'netTOAcs':{'vars':['netTOAcs'],'signs':[1.0]},
        'pr':{'vars':['pr'],'signs':[1.0]}
        
        #'netSurf':{'vars':['netSurf'],'signs':[1.0]},
        #'netSurfCRE':{'vars':['cresSurf','crelSurf'],'signs':[1.0,1.0]},
        }
  
'''
:{'vars':[],'signs':[]},    

'''
CMIPvarDefs={
        'netTOA':{'vars':['rsdt','rlut','rsut'],'signs':[1.0,-1.0,-1.0]},
        'netTOAcs':{'vars':['rsdt','rlutcs','rsutcs'],'signs':[1.0,-1.0,-1.0]},
        'crel':{'vars':['rlut', 'rlutcs'],'signs':[-1.0,1.0]},
        'cres':{'vars':['rsut','rsutcs'],'signs':[-1.0,1.0]},        
        'netTOACRE':{'vars':['rlut', 'rlutcs','rsut','rsutcs'],'signs':[-1.0,1.0,-1.0,1.0]},
        'pr':{'vars':['pr'],'signs':[1.0]}
        #'netSurf':{'vars':['rsds','rlds','rsus','rlus','hfss','hfls'],'signs':[1.0,1.0,-1.0,-1.0,-1.0,-1.0]},
        #'netSurfCRE':{'vars':['cresSurf','crelSurf'],'signs':[1.0,1.0]},
        }
     
    
### calculate vars
def calERA5(varName,months):
    i=0
    for t,s in zip(ERAvarDefs[varName]['vars'],ERAvarDefs[varName]['signs']):
        
        if(i==0):
            varOut=ERAInput[t]*s
        else:
            varOut=varOut+ERAInput[t]*s
        i=i+1
    return varOut.where(varOut.time.dt.month.isin(months), drop=True)

def calCMIP(varName,months,modelName):
    DataInput=xr.open_dataset(Inpath+modelName+'_historical.nc').sel(time=slice(startD,endD))
    i=0
    for t,s in zip(CMIPvarDefs[varName]['vars'],CMIPvarDefs[varName]['signs']):
        
        if(i==0):
            varOut=DataInput[t]*s
        else:
            varOut=varOut+DataInput[t]*s
        i=i+1
    return varOut.where(varOut.time.dt.month.isin(months), drop=True)

def chkVarAvail(varName,modelName):
    DataInput=xr.open_dataset(Inpath+modelName+'_historical.nc').sel(time=slice(startD,endD))
    flag=True
    for t in CMIPvarDefs[varName]['vars']:
        flag=t in DataInput
        #print(t,flag)
        if not flag:
            break
    
    return flag

In [ ]:
################ get JJAS(1979-2014) ISMR(10-30N 60-100E) RMSE (model - era) (area weighted by lat) for each model 
prBiasList=[]
varN='pr'
months=[6,7,8,9]

for i,f in enumerate(modelList['fName']):
    #
    flag=chkVarAvail(varN,f)
    if flag:
        bias=calCMIP(varN,months,f).sel(lat=slice(10,30),lon=slice(60,100)).mean(dim='time')*86400.0-calERA5(varN,months).sel(lat=slice(10,30),lon=slice(60,100)).mean(dim='time')*86400.0
        ### square the error
        sBias=np.sqrt(bias*bias)
        ### weighted mean of squared error
        weights=np.cos(np.deg2rad(sBias.lat))
        msBias=sBias.weighted(weights).mean(dim=('lat','lon'))
        prBiasList.append(msBias.values)
#prBiasList     
prBiasHigh=np.mean(prBiasList)+2*np.std(prBiasList)
prBiasHigh

## Xarray of model rmse
prRMSE=xr.DataArray(data=prBiasList,coords={'models':np.arange(1,len(modelList)+1)})
#prRMSE

In [ ]:
def getRadBias(varN):
    BiasList=[]
    for i,f in enumerate(modelList['fName']):
        flag=chkVarAvail(varN,f)
        if flag:
            bias=calCMIP(varN,months,f).mean(dim='time')-calERA5(varN,months).mean(dim='time')
            BiasList.append(bias)
    ## Xarray of model rmse
    RadRMSE=xr.DataArray(data=BiasList,coords={'models':np.arange(1,len(modelList)+1),'lat':BiasList[0].lat,'lon':BiasList[0].lon})
    prRMSE
    return RadRMS

def linear_reg_slope(x, y):
    pf = np.polyfit(x, y, 1)
    #print(pf)
    return xr.DataArray(pf[0])

def linear_reg_intercep(x, y):
    pf = np.polyfit(x, y, 1)
    #print(pf)
    return xr.DataArray(pf[1])
#,xr.DataArray(pf[1])

In [ ]:
varList=['netTOA','cres','crel','netTOAcs','netTOACRE']
fig,axes = pplt.subplots([[1,2],[3,4],[5,6]                          
                         ],proj='eqearth')

for i,vname in enumerate(varList):
    RadBias=getRadBias(vname)
    slope = xr.apply_ufunc(linear_reg_slope,
                        prRMSE, RadBias,
                        vectorize=True,
                        input_core_dims=[['models'], ['models']],# reduce along 'plev'
                        )
    intercept = xr.apply_ufunc(linear_reg_intercep,
                        prRMSE, RadBias,
                        vectorize=True,
                        input_core_dims=[['models'], ['models']],# reduce along 'plev'
                        )
    #print(slope)
    #print(intercept)
    plotReg=slope*prBiasHigh+intercept
    lons,lats = np.meshgrid(plotReg.lon,plotReg.lat)
    con = axes[i].contourf(lons,lats,plotReg,levels=np.arange(-30.0,33,3),extend='both',colorbar_kw={'label': 'Wm-2'})
    axes[i].set_title(vname)
axes.format(suptitle='RMSE of variables regressed on RMSE of SASM rainfall (JJAS climatology 1979-2014) ')
fig.format(coast=True)
fig.colorbar(con)
